In [1]:
%matplotlib inline

In [2]:
import pandas
import os.path
import pkg_resources
import sqlalchemy
import sqlalchemy.orm
import skopy.features
import sklearn.decomposition
import sklearn.preprocessing

In [3]:
directory = pkg_resources.resource_filename("skopy", "data")

In [4]:
connection = "sqlite:///{}/measurements.sqlite".format(directory)

In [5]:
engine = sqlalchemy.create_engine(connection)

skopy.features.Base.metadata.create_all(engine)

session = sqlalchemy.orm.sessionmaker()

session.configure(bind=engine)

session = session()

In [6]:
metadata = pandas.read_csv("../metadata.csv")

In [7]:
for _, record in metadata.iterrows():
    x_pathname = os.path.join(directory, record["image"])
    y_pathname = os.path.join(directory, record["label"])
  
    image = skopy.features.Image(x_pathname, y_pathname)

    session.add(image)

In [8]:
session.commit()

In [9]:
images = pandas.read_sql_table("images", connection)

In [10]:
instances = pandas.read_sql_table("instances", connection)

In [11]:
features = pandas.merge(images, instances, left_on="id", right_on="image_id")

column_names = [
    "id_x",
    "id_y",
    "image_id",
    "image_pathname",
    "index",
    "label_pathname",
    "pathname_x",
    "pathname_y"
]

metadata = features[column_names]

dropped_column_names = [
    "id_x", 
    "id_y", 
    "image_pathname", 
    "label_pathname"
]

metadata = metadata.drop(dropped_column_names, 1)

metadata.columns = ["id", "index", "image", "mask"]

features = features.drop(column_names, 1)

features.columns = features.columns.str.replace("(.*)_x", "x_\g<0>")
features.columns = features.columns.str.replace("(.*)_y", "y_\g<0>")

features.columns = features.columns.str.replace("_x", "")
features.columns = features.columns.str.replace("_y", "")

In [13]:
imputed = features.dropna()

In [20]:
scaled = sklearn.preprocessing.StandardScaler()

scaled.fit(imputed)

scaled = standard_scaler.fit_transform(imputed)

scaled = pandas.DataFrame(scaled, columns=imputed.columns)

In [22]:
scaled.shape

(631, 808)